#### План отчета:
&nbsp;

**1.	Постановка задачи и пути решения.**

&#8195;Зачем нужно? Почему важно? Как решать?


**2.	Первичный анализ данных**

&#8195;Что имеем? Стратегия работы

**3.	Предобработка выборки**

&#8195;Как преобразуем выборку что бы следовать стратегии работы?
>Можно выполнить сравнение (с применением виджетов) работы функции, считающей признаки для выборки. С добавлением значений по одному в соответствующую ячейку dataframe и добавлением по-столбцам (разница ~20 раз)

**4.	Проверка гипотез**

&#8195;Какие показатели значимо выделают одних пользователей от других? Почему?

**5.	Features development**

&#8195;Какие фичи будем пробовать? Почему? Что получилось?

**6.	Обучение и каггл**

&#8195;Почему бинарная классификация показывает себя хорошо, а многоклассовая дает плохие показатели метрик. Сравим метрики на каггле при разном количестве признаков

**7.	Многоклассовая классификация с Ворпал вэббит и SGDlogreg**

&#8195;Почему это в принципе так неэффективно и что менее неэффективно.

**8.	Вывод по решению задачи на данный момент, возможности ее выкатывания в продакшн, а так же дальнейшая стратегия по исследованию данной темы.**


### Содержание
&nbsp;
1. [Цель и задачи проекта. Описание исходных и обработанных данных и признаков](#1part)
2. [Подготовка данных](#2part)
  - [Создание сессий](#2.1part)
  - [Применение мешка слов](#2.2part)
  - [Применение разных длин сессий и идеи скользящего окна](#2.3part)
3. [Первичный анализ данных. Проверка гипотез](#3part)
4. [Разработка признаков и их визуальный анализ](#4part);
5. Подбор модели;
6. Итоги, выводы и описание дальнейшей работы

<a id='1part'></a>
### 1. Цель и задачи проекта. Описание исходных и обработанных данных и признаков
&nbsp;

1.1 Цель проекта

&#8195;Проект направлен на решение задачи идентификации пользователя по его поведению в сети Интернет. Задача как таковая является широкой в смысле выбора используемых признаков и стратегии. К примеру идентифицировать пользователя можно как по тому какие сайты в какое время он посещает, так и по тому как он это делает - какими разделами сайта он пользуется, как быстро отвечает на сообщения в социальных сетях, насколько длинны оставляемые им комментарии, или он предпочитает их не оставлять и т.д.

&#8195;Целью данного проекта является построение алгоритма идентификации пользователя по имеющимся данным с последовательностями посещенных пользователями сайтов и времени посещения каждого из них.

&nbsp;

1.2 Задачи проекта

- Предобработка данных, подготовка их к классификации
- Первичный и визуальный анализ
- Разработка признаков и их описание
- Сравнение семейств моделей классификации и подбор параметров выбранной модели
- Оценка метрик и вывод по проделанной работе
- Прогноз по дальнейшей работе


<a id='2part'></a>
### 2. Подготовка данных и первичный анализ
&nbsp;



&#8195;Имеются данные с прокси-серверов Университета Блеза Паскаля.

&#8195;Для каждого пользователя заведен csv-файл с названием user****.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: 

&nbsp;

<center>*timestamp, посещенный веб-сайт*</center>
&nbsp;

&#8195;В дальнейшем будет использоваться параллельно 2 выборки: по 10 и по 150  пользователям. Для 10 пользователей будет отлаживаться код, для 150 будет использоваться рабочая версия.

&#8195;Вид исходных данных приведен ниже. 

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-yw4l">site</th>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:07</td>
    <td class="tg-yw4l">fpdownload2.macromedia.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.google.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:18</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
</table>

&#8195;На данном этапе поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. 

&#8195;Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов. Целевым классом будет id пользователя.

&#8195;Отображение сайтов в их индексы выглядит так:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>


<a id='2.1part'></a>
&#8195;Для подготовки обучающей выборки реализована функция  prepare_train_set, которая принимает на вход путь к каталогу с csv-файлами path_to_csv_files и параметр session_length – длину сессии, а возвращает 2 объекта: 
- DataFrame, в котором строки соответствуют уникальным сессиям из session_length сайтов, session_length столбцов – индексам этих session_length сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}

После применения функции получаем таблицу вида:

&nbsp;



<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>user_id</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>3</td>
      <td>2</td>
      <td>2</td>
      <td>9</td>
      <td>2</td>
      <td>1</td>
      <td>7</td>
      <td>5</td>
      <td>8</td>
      <td>10</td>
      <td>0001</td>
    </tr>
    <tr>
      <th>1</th>
      <td>3</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0001</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>2</td>
      <td>6</td>
      <td>6</td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0002</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>1</td>
      <td>5</td>
      <td>11</td>
      <td>4</td>
      <td>0003</td>
    </tr>
    <tr>
      <th>4</th>
      <td>4</td>
      <td>1</td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0003</td>
    </tr>
  </tbody>
</table>

И словарь вида:

     {'google.com': [1, 9],
     'oracle.com': [2, 8],
     'vk.com': [3, 3],
     'meduza.io': [4, 3],
     'mail.google.com': [5, 2],
     'football.kulichki.ru': [6, 2],
     'accounts.google.com': [7, 1],
     'apis.google.com': [8, 1],
     'geo.mozilla.org': [9, 1],
     'plus.google.com': [10, 1],
     'yandex.ru': [11, 1]}

<a id='2.2part'></a>
&#8195;Далее, что бы наши категориальные признаки с индексами сайтов приобрели смысл в рамках машинного обучения применим идею мешка слов. Точнее говоря не только идею, но и фактически преобразуем матрицы признаков в матрицу, где число столбцов равно количеству уникальных индексов сайтов, а в ячейках указано количество посещений того или иного сайта в рамках сессии.

&#8195;Делать это будем с помощью разреженных матриц Scipy – csr_matrix. 

&#8195;Напишем функцию sparsecreator, которая приобразовывает массив с категориальными признаками в разряженную матрицу с вещественными.

&#8195;Для 150 пользователей получим матрицу размером 137019x27797, где 27797 - число уникальных сайтов в выборке.

&#8195;Полученная матрица:

    matrix([[4, 2, 0, ..., 0, 0, 0],
            [0, 1, 0, ..., 0, 0, 0],
            [5, 1, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

<a id='2.3part'></a>
&#8195;Пока мы брали последовательности из 10 сайтов. Далее сделаем число сайтов в сессии параметром, так что бы в дальнейшем сравнить модели классификации, обученные на разных выборках – с 5, 7, 10 и 15 сайтами в сессии. А так же применим идею скользящего окна – сессии будут перекрываться.

&#8195;Разработаем функцию prepare_sparse_train_set_window.

&#8195;Аргументы:
- path_to_csv_files – путь к каталогу с csv-файлами
- site_freq_path – путь к pickle-файлу с частотным словарем, полученным в 1 части проекта
- session_length – длина сессии (параметр)
- window_size – ширина окна (параметр)

&#8195;Значения:
- разреженная матрица X_sparse (двухмерная Scipy.sparse.csr_matrix), в которой строки соответствуют сессиям из session_length сайтов, а max(site_id) столбцов – количеству посещений site_id в сессии.
- вектор y (Numpy array) "ответов" в виде ID пользователей, которым принадлежат сессии из X_sparse

&#8195;Запустим созданную функцию со значениям параметра session_length (15, 10, 7, 5) и значениям параметра window_size (10, 7, 5) для выборки 10 и 150 пользователей. Сериализуем полученные 16 разреженных матриц (обучающие выборки) и вектора с метками целевого класса в файлы.

<a id='3part'></a>
### 3. Первичный анализ данных. Проверка гипотез


&#8195;Проверка гипотез будет осуществляться на матрице сессии-сайты с длинной сессии и окна 10 для выборки 10 пользователей.

&#8195;Так выглядит ее структура:

&nbsp;
    
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site1</th>
      <th>site2</th>
      <th>site3</th>
      <th>site4</th>
      <th>site5</th>
      <th>site6</th>
      <th>site7</th>
      <th>site8</th>
      <th>site9</th>
      <th>site10</th>
      <th>user_id</th>
    </tr>
    <tr>
      <th>session_id</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>192</td>
      <td>576</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>133</td>
      <td>3</td>
      <td>133</td>
      <td>203</td>
      <td>133</td>
      <td>31</td>
    </tr>
    <tr>
      <th>1</th>
      <td>421</td>
      <td>193</td>
      <td>677</td>
      <td>254</td>
      <td>133</td>
      <td>31</td>
      <td>393</td>
      <td>3403</td>
      <td>217</td>
      <td>55</td>
      <td>31</td>
    </tr>
    <tr>
      <th>2</th>
      <td>55</td>
      <td>3</td>
      <td>55</td>
      <td>55</td>
      <td>5</td>
      <td>293</td>
      <td>421</td>
      <td>333</td>
      <td>899</td>
      <td>55</td>
      <td>31</td>
    </tr>
    <tr>
      <th>3</th>
      <td>473</td>
      <td>3570</td>
      <td>473</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>55</td>
      <td>946</td>
      <td>200</td>
      <td>123</td>
      <td>31</td>
    </tr>
    <tr>
      <th>4</th>
      <td>343</td>
      <td>55</td>
      <td>5</td>
      <td>3399</td>
      <td>258</td>
      <td>212</td>
      <td>3498</td>
      <td>2095</td>
      <td>674</td>
      <td>2095</td>
      <td>31</td>
    </tr>
  </tbody>
</table>

&#8195;Распределение целевого класса:
    
    128    2796
    39     2204
    207    1868
    127    1712
    237    1643
    33     1022
    50      802
    31      760
    100     720
    241     534
    Name: user_id, dtype: int64
    
&#8195;Посчитаем распределение числа уникальных сайтов в каждой сессии из 10 посещенных подряд сайтов:

    7     2308
    6     2197
    8     2046
    5     1735
    9     1394
    2     1246
    4     1163
    3      894
    10     651
    1      427
    dtype: int64
![](image1.png)

&#8195;Для проверки нормальности рапределения числа уникальных сайтов воспользуемся критерием Шапиро-Уилка и построим QQ-график.
 ![](image2.png)
 
&#8195;Критерий на данной выборке показывает p-value близкое к нулю, на QQ-графике видны длинные хвосты, т.е. имеется перекос в сторону больших и минимальных значений. Число уникальных сайтов распределено не нормально.

&#8195;Проверим с помощью биномиального критерия для доли, что доля случаев, когда пользователь повторно посетил какой-то сайт (то есть число уникальных сайтов в сессии < 10) велика: больше 95%.

&#8195;Полученное p-value 0.022

&#8195;95% доверительный интервал Уилсона для доли случаев, когда пользователь повторно посетил какой-то сайт:

(0.9501028841411286, 0.9570527377232229)

На основании этих результатов можно уверенно предположить, что пользователи склонны посещать сайты (сайт) повторно в рамках сессии.

Постром распределение частоты посещения сайтов для сайтов, которые были посещены как минимум 1000 раз.
 ![ss](image3.png)

Построим 95% доверительный интервал для средней частоты появления сайта в выборке на основе bootstrap. Используем столько же bootstrap-подвыборок, сколько сайтов оказалось в исходной выборке по 10 пользователям (получим массив 4913 Х 4913).

Полученный интервал средней частоты появления сайта в выборке: (22.51524527, 35.76303684)

<a id='4part'></a>
### 4. Разработка признаков и их визуальный анализ